In [61]:
import re
import os

def extract_data_from_txt(file_path):
    """
    Extract bounding box parameters, label, and image name from a given .txt file.
    
    Parameters:
    - file_path: path to the .txt file.
    
    Returns:
    - List of tuples with format (img_name, x_min, y_min, x_max, y_max, label).
    """
    
    with open(file_path, "r") as file:
        lines = file.readlines()
    
    # Extract image name from the file path
    img_name = os.path.basename(file_path).replace("2021", "").split("_")[0]
    
    # List to store extracted data
    data = []
    h, w = map(int, lines[-1].strip().split(": ")[1].split(", "))
    for line in lines:
        if "BBOX" in line:
            # Extract bbox coordinates
            bbox = re.search(r"tensor\(\[(.+?)\]\)", line).group(1).split(", ")
            x_min, y_min, x_max, y_max = map(float, bbox)
            
            # Extract label
            label = re.search(r"LABEL: (.+?) \(ID", line).group(1)

            data.append((str(img_name), x_min, y_min, x_max, y_max, label, "\"" + str((h, w)) + "\""))
    
    return data



In [63]:
import os

directory_path = "output/from_dataset/filtered_unknown/SOWODB_backup/"
desired_suffix = "gt.txt"

# List all files in the directory
all_files = os.listdir(directory_path)

# Filter the files based on the desired suffix
filtered_files = [f for f in all_files if f.endswith(desired_suffix)]

print(len(filtered_files))


4952


In [64]:
# List to store all the extracted data from the files
all_data = []

# Extract data from all the files
for path in filtered_files:
    all_data.extend(extract_data_from_txt(directory_path + path))

# Convert the extracted data into a CSV format
csv_output_path = "bbox_data_unscaled_gt.csv"

# Sort all_data based on the image_id before writing to the CSV file
all_data.sort(key=lambda x: int(x[0]))

# Writing the combined data to a CSV file
with open(csv_output_path, "w") as csv_file:
    # Writing the header
    csv_file.write("img_name,x_min,y_min,x_max,y_max,label, scaled_size\n")
    
    # Writing the data
    for row in all_data:
        csv_file.write(",".join(map(str, row)) + "\n")

# 
csv_output_path


'bbox_data_unscaled_gt.csv'

In [65]:
import pandas as pd
import os
from PIL import Image

def get_actual_dimensions(image_id, image_dir):
    """
    Get the actual dimensions (width, height) of an image given its image_id.

    Parameters:
    - image_id: ID/name of the image (without file extension).
    - image_dir: Directory containing the images.

    Returns:
    - Tuple (width, height) of the actual image dimensions.
    """
    image_path = os.path.join(image_dir, image_id + ".jpg")
    
    # Check if the image exists, if not, drop the first 2 digits and try again
    if not os.path.exists(image_path):
        image_id = image_id[2:]
        image_path = os.path.join(image_dir, image_id + ".jpg")
    
    with Image.open(image_path) as img:
        return img.size

def scale_bbox(row, image_dir):
    # Get actual image dimensions
    actual_width, actual_height = get_actual_dimensions(row['img_name'], image_dir)

    # Store the actual size as a tuple-style string
    row['actual_size'] = f"({actual_height}, {actual_width})"
    
    # Get scaled dimensions from the CSV
    scaled_height, scaled_width = tuple(map(int, row[' scaled_size'][1:-1].split(',')))
    
    # Compute scaling factors
    x_scale = actual_width / scaled_width
    y_scale = actual_height / scaled_height
    
    # Scale bounding box coordinates
    row['x_min'] *= x_scale
    row['y_min'] *= y_scale
    row['x_max'] *= x_scale
    row['y_max'] *= y_scale
    
    return row

image_dir = "/training_data_2/yuetian/OWOD/JPEGImages"
scaled_data =  pd.read_csv('bbox_data_unscaled_gt.csv', dtype={'img_name': str})

# Apply the scaling function to each row of the DataFrame
scaled_data = scaled_data.apply(lambda row: scale_bbox(row, image_dir), axis=1)
scaled_data.to_csv('bbox_data_scaled_gt.csv', index=False)
scaled_data.head()


,img_name,x_min,y_min,x_max,y_max,label,scaled_size,actual_size
0,00000018,70.656254,14.250000,112.812507,112.416654,unknown,"(750, 1333)","(285, 380)"
1,00000018,80.156254,231.958308,108.656254,273.125000,unknown,"(750, 1333)","(285, 380)"
2,00000018,29.687493,167.041654,98.562507,278.666654,person,"(750, 1333)","(285, 380)"
3,00000018,98.562507,250.166692,108.656254,272.333346,person,"(750, 1333)","(285, 380)"
4,00000018,88.468761,54.625000,180.499971,281.041654,person,"(750, 1333)","(285, 380)"


In [66]:
scaled_data = scaled_data[scaled_data['img_name'].str.len() >= 10]
scaled_data.to_csv('bbox_data_scaled_gt.csv', index=False)

scaled_data

,img_name,x_min,y_min,x_max,y_max,label,scaled_size,actual_size
5,000000000139,235.999973,141.999965,262.000013,213.000000,unknown,"(800, 1201)","(426, 640)"
6,000000000139,6.999980,166.999988,156.999993,262.999993,unknown,"(800, 1201)","(426, 640)"
7,000000000139,556.999940,208.999966,638.999980,287.999962,unknown,"(800, 1201)","(426, 640)"
8,000000000139,358.000013,217.999962,414.999980,320.999999,unknown,"(800, 1201)","(426, 640)"
9,000000000139,289.999987,217.999962,353.000020,317.000019,unknown,"(800, 1201)","(426, 640)"
...,...,...,...,...,...,...,...,...
36776,000000581781,581.999985,140.999962,641.000037,228.999941,unknown,"(800, 1071)","(478, 640)"
36777,000000581781,74.999993,104.999990,173.999985,213.000027,unknown,"(800, 1071)","(478, 640)"
36778,000000581781,2.000015,2.000012,45.000007,172.999971,unknown,"(800, 1071)","(478, 640)"
36779,000000581781,192.999978,183.000030,443.000052,246.999987,unknown,"(800, 1071)","(478, 640)"
